# Exploration by Random Network Distillation

**Abstract**
<br>
exploration bonus 를 활용하여 1. easy to implement, 2. minimal overhead to computation 한 딥러닝 기법을 소개합니다.
> The bonus is error of a neural network predicting features of the observations given by a fixed randomly initializd neural network.
> flexibly combine intrinsic and extrinsic rewards
<br>

우리는 random network distillation (RND) bonus with increased flexibility 을 통하여 exploration 이 많이 필요한 몇 몇 atari 게임들에 있어 의미있는 진전을 이루었습니다. 특히 심층 강화학습 기법들에서 가장 어렵다고 알려진 Montezuma's Revenge 라는 게임에 있어 최고의 성능을 보였으며, 어떠한 데모를 통한 학습이나 상태의 접근 없이 전문 게임 플레이어 보다 나은 점수를 얻은 첫 번째 기법입니다.


## 1. Introduction
강화학습 기법들은 expected return of policy 를 최대화 하는 방법으로 동작합니다. 그리고 이러한 방법은 환경이 dense rewards 를 가질 수 있을 때에는 잘 동작하지만, sparse 하거나 찾기 힘들 때에는 실패하기 쉽습니다. 현실 세계에서 강화학습 에이전트가 문제를 해결하기에 dense reward functions 을 매번 만들어 내는 것은 실용적이지 않을 것입니다. 이러한 상황에서 환경을 탐험하는 직접적인 방법이 필요합니다.
![Figure.1](images/figure1.png)

 최근 강화학습 가장 도전적인 일들을 해결하는 데에 추천되고 있으며, 병렬로 실행되는 환경에서 대용량 데이터의 처리를 필요로 합니다. 이러한 점을 비추어 보았을 때에 대용량 데이터 처리가 가능한 확장성 있는 탐험 기법들이 필요로 하는데, 최근에 소개된 많은 기법(based on counts, pseudo-counts, information gain or prediction gain)들은 병렬 환경의 대용량 처리에는 확장성이 떨어집니다.

 본 논문에서 소개하는 단순하면서도 구현하기 쉬운 탐험 보너스 기법은 고차원 observations 에서도 잘 동작하며, policy optimization 알고리즘도 적용할 수 있으며, 배치 처리에서 단지 single forward pass of neural network 만을 필요로 하므로 아주 효율적입니다.
  우리의 탐사 보너스는 신경망이 훈련 된 것과 유사한 예에서 예측 오차가 현저하게 낮은 경향이 있다는 관찰에 기반하며, 이러한 사실은  에이전트가 과거에 경험을 학습한 네트워크가 새로운 경험의 참신함을 계량화 하는 데에 예측 오류를 활용할 수 있게 되었다.
  
  많은 저자들이 언급하였지만 예측오류를 최대화 하는 에이전트들은 입력 데이터들에 대한 확률적 함수와 같이 transitions 하기 쉽게 되는데, 예를 들어 현재 observation과 에이전트의 action 에 대한 다음 observation 을 에측하는 문제가 있다고 한다면, 에이전트는 예측 오류를 줄이기 위해 stochastic transitions 을 찾으려고 할 것이고 일정하고 랜덤하게 변화하는 noise TV 와 같은 혹은 coin tosses 등에 쉽게 휘말리게 된다. 이러한 observation 들을 확인함으로써, 예측의 오류가 아니라 상대적인 개선을 정량화 할 수 있는 기법에 대한 사용하는 데에 동기부여를 할 수가 있었다. 하지만 이러한 기법들은 효과적인 구현이 아주 어렵다.
  
 위에서 언급한 정답이 입력에 대한 deterministic 함수인 문제를 예측하는 데에 탐사 보너스를 정의함으로써 발생하는 의도하지 않은 확률론적인 것에 대한 대안을 제시합니다.  랜덤 초기화된 고정된 신경망에 현재의 observation 을 입력으로 한 output 을 예측하는 방법이다.
 
 <중략>
 
 extrinsic rewards 를 같이 사용하지 않더라도 RND 탐사 보너스만으로 몬테주마 복수 게임의 절반 이상의 방을 찾을 수 있었다. 탐사 보너스와 extrinsic rewards 를 통합하기 위해서 extrinsic, intrinsic rewards 의 2가지 reward streams 를 적용한 PPO 알고리즘을 소개하였습니다. 이러한 방법으로 episodic and non-episodic returns 를 통합할 수 있었꼬, 2가지 rewards 에 대하여 별도의 discount rates 를 적용할 수가 있었습니다.
> episodic 은 해당 에피소드 당 얻는 extrinsic 관련 reward 이고 non-episodic 은 현재 에피소드 별 상황과는 무관하게, random network 와 predict network 를 통해 observation 예측에 따른 error 를 reward 를 말한다.
 

## 2. Method
### 2.1 Exploration bonuses
환경 보상이 너무 sparse 한 경우 에이전트에게 탐험을 하도록 북돋우는 종류 중에 하나이다.

#### Count-based exploration - 자주 방문하는 곳 보다 novel state 의 보상이 더 커야 한다
##### 1. Tabular case : $i_t = 1/{n_t(s)}\ and\ i_t = 1/\sqrt{n_t(s)} \ - \ n_t(s)$ : 상태 s 의 방문 횟수를 활용
##### 2. Non-tabular case : 모든 state 를 방문하기 어렵기 때문에 상태의 밀도 추정의 변화를 pseudo-count 로 활용 할 수 있는데 이전에 방문한 적이 없더라도 유사한 곳은 긍정정인 보상을 반환하는 방식이다
##### 3. Generic prediction problem : pred. error w/ agent's transitions : forward dynamics and inverse dynamics
##### 4. Non-generic prediction problem

## 2.2 Random Network Distillation
이 논문에서는 prediction problem 이 드물게 생성되는 다른 접근을 하겠다
#### 2개의 신경망을 사용
##### 1. 랜덤 초기화된 고정 _target network_ : 해결해야 하는 예측 문제
##### 2. 에이전트의 수집된 데이터에 의해 학습된 _predictor netwrok_ : 랜덤 네트워크와의 예측 오류를 최소화하도록 학습
> 1. 2개의 네트워크는 observation 에 대한 reward 값의 MSE 를 최소화 하도록 gradient descent 기법에 의하여 학습되며 paramters $\theta_{\hat f}$ 를 따른다.
> 2. 이러한 과정은 랜덤 초기화된 신경망에서 훈련된 신경망으로 정수만 빼내게 합니다. 
> 3. 예측 오류는 predictor 가 학습해왔던 것들과는 다른 novel states 에서 더욱 높게 나타나는 것을 예상할 수 있습니다.
> 4. _predictor 가 학습해왔던 것들_은 늘 하던 행동에 의해 학습된 모델을 말하고, _novel states_ 는 평소에 하지 않았던 우연히 잘 한 행동을 말합니다.
<br>

#### 직관적인 확인을 위해 MNIST 적용
##### 1. 랜덤 초기화된 target network 를 따라하는 predictor 신경망을 훈련시킵니다
> 0을 학습하기 위한 예제 데이터를 말하며 아마도 다른 숫자도 포함은 되지만 0만 학습하기 위한 one-hot encoding 처럼 보입니다

##### 2. 새로운 예제를 통해 predictor network 테스트를 하고 MSE를 측정합니다
> 1. target class는 자주 방문하지 않은 state 이며 새로운 obs 들을 발생시킬 것입니다
> 2. 결국 Figure2 에서 보다시피 predictor network 는 초기에는 아주 높은 prediction error 를 보여주어 intrinsic reward 에 효과적인 지표로 사용할 수 있고 학습이 끝나갈 수록 점점 줄어들고 있어 novelty 인지에 좋다는 것을 보여줍니다.
> 3. 참고로, 학습에 필요한 네트워크는 총 3개이며 (0: static random network, 1: predictor network, 2: classifier network for RL)
<br>

![novelty detection](images/figure2.png)
##### 3. Figure1 에서 novel state 에서는 intrinsic reward 가 높았던 것을 볼 수 있습니다.


## Q&A
### 1. state density estimation?
### 1. forwrad dynamics?